In [1]:
import numpy as np

# Value Loss

In [ ]:
def r_gamma(results: np.array, gamma):
    r_gamma = results[0]
    for result in results[:0:-1]:  # Slicing to reverse except the first element
        r_gamma = gamma * (r_gamma + result)
    return r_gamma

def value_function(state): #TODO: implement this
    return 0

def value_loss(r_gamma, state, deltas):
    return (np.clip(r_gamma, -deltas[2], deltas[3])-value_function(state))**2

# Policy Loss

In [ ]:
def ratio(old_policy, new_policy, action, state):
    return new_policy(action, state) / old_policy(action, state)

def bellman_residuals(states, actions, rewards, deltas, gamma):
    pass

def advantage_function(k, results, value_function: callable, gamma):
    pass

def tc_loss_function(ratio, advantage, epsilon, deltas):
    return np.clip(ratio, np.clip(ratio, 1 - epsilon, 1 + epsilon), deltas[1]) * advantage
    
